<a href="https://colab.research.google.com/github/suraj0693/my_own_gpt/blob/main/MyOwnGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("nampdn-ai/mini-en")

In [ ]:
len_text = [len(a) for a in dataset['train']['text']]

In [ ]:
max_length = max(len_text)

In [ ]:
max_length

7998

In [ ]:
min_length = min(len_text)

In [ ]:
min_length

256

In [ ]:
text = []
for a in dataset['train']['text']: text.append(a[:512])

In [ ]:
text[0].split(" ")[:1]

['By']

In [ ]:
inp = []
out = []
for j, t in enumerate(text):
  _inp = []
  words = ['<s>']+t.split(' ')+['</s>']
  for i in range(1,len(words)):
    _inp.append(words[i-1])
    for_inp = _inp.copy()
    inp.append(for_inp)
    out.append(_inp+[words[i]])
  if j>1:
    break


In [ ]:
for i in range(len(inp)):
  print(inp[i], out[i])

In [ ]:
vocab = [t for _text in text for t in _text.split(' ')]

In [ ]:
vocab = list(set(vocab))

In [ ]:
len(vocab)

2128018

In [ ]:
K = torch.rand(size=[5,5], requires_grad=True)
Q = torch.rand(size=[5,5], requires_grad=True)
V = torch.rand(size=[5,5], requires_grad=True)

In [ ]:
input = torch.rand(size=[1,5], requires_grad=False)

In [ ]:
k = input*K
q = input*Q
v = input*V

In [ ]:
kq = []
for i in range(5):
  kq.append(torch.dot(k[i],q[i]))
#kq = kq/(5)**0.5
kq = torch.tensor(kq)
kq = kq/(5)**0.5
kq = torch.nn.functional.softmax(kq)

<ipython-input-101-d124a7676c0c>:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  kq = torch.nn.functional.softmax(kq)


In [ ]:
kq

tensor([0.2163, 0.1733, 0.1773, 0.1911, 0.2421])

In [ ]:
weighted_values = []
for attention, value in zip(kq,v):
  weighted_values.append(value*attention)

In [ ]:
weighted_values

[tensor([0.1665, 0.0140, 0.0342, 0.1305, 0.0528], grad_fn=<MulBackward0>),
 tensor([0.0090, 0.0009, 0.0683, 0.0522, 0.0336], grad_fn=<MulBackward0>),
 tensor([0.0382, 0.0076, 0.0351, 0.0043, 0.0588], grad_fn=<MulBackward0>),
 tensor([0.1012, 0.0135, 0.0169, 0.0574, 0.0030], grad_fn=<MulBackward0>),
 tensor([0.1072, 0.0124, 0.0318, 0.1001, 0.0881], grad_fn=<MulBackward0>)]

In [ ]:
sum(weighted_values)

tensor([0.4222, 0.0484, 0.1862, 0.3445, 0.2363], grad_fn=<AddBackward0>)

In [ ]:
q[0]

tensor([0.1578, 0.0384, 0.0930, 0.9454, 0.2076], grad_fn=<SelectBackward0>)

In [ ]:
k[0]

tensor([0.0496, 0.0114, 0.3533, 0.4584, 0.1715], grad_fn=<SelectBackward0>)

In [ ]:
class SelfAttention:

  def __init__(self, dimension):
    self.d = dimension

  def __call__(self, input):
    K = torch.rand(size=[self.d,self.d], requires_grad=True)
    Q = torch.rand(size=[self.d,self.d], requires_grad=True)
    V = torch.rand(size=[self.d,self.d], requires_grad=True)
    k = input*K
    q = input*Q
    v = input*V
    kq = []
    for i in range(self.d):
      kq.append(torch.dot(k[i],q[i]))
    kq = torch.tensor(kq)
    kq = kq/(self.d)**0.5
    kq = torch.nn.functional.softmax(kq)

    weighted_values = []
    for attention, value in zip(kq,v):
      weighted_values.append(value*attention)
    return sum(weighted_values)


In [ ]:
input = torch.rand(size=[1,10])
print(input)
attn = SelfAttention(10)
out = attn(input)

tensor([[0.9096, 0.2932, 0.0607, 0.6927, 0.5236, 0.1401, 0.1287, 0.7583, 0.6158,
         0.3377]])


<ipython-input-92-652908cec684>:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  kq = torch.nn.functional.softmax(kq)


In [ ]:
out[7:]=-np.inf

In [ ]:
out

tensor([0.2722, 0.1385, 0.0291, 0.3491, 0.2432, 0.0755, 0.0639,   -inf,   -inf,
          -inf], grad_fn=<CopySlices>)

In [ ]:
import numpy as np
-np.inf

-inf

In [ ]:
class MaskedAttention:
  def __init__(self, dimension):
    self.d = dimension

  def __call__(self, input):
    K = torch.rand(size=[self.d,self.d], requires_grad=True)
    Q = torch.rand(size=[self.d,self.d], requires_grad=True)
    V = torch.rand(size=[self.d,self.d], requires_grad=True)
    k = input*K
    q = input*Q
    v = input*V
    kq = []
    for i in range(self.d):
      kq.append(torch.dot(k[i],q[i]))
    kq = torch.tensor(kq)
    kq = kq/(self.d)**0.5
    attention = torch.nn.functional.softmax(kq)

    i = 0
    for inp in input:
      if inp==torch.tensor([0]*self.d):
        break
      else:
        i+=1
    attention[i:]=-np.inf

    weighted_values = []
    for attn, value in zip(attention,v):
      weighted_values.append(value*attn)
    return sum(weighted_values)

In [ ]:
class FullyConnectedLayer:
  pass

In [ ]:
class Dropout:
  pass

In [ ]:
class NormalizationLayer:
  pass